# Agents

AutoGen AgentChat provides a set of preset Agents, each with variations in how an agent might respond to messages. All agents share the following attributes and methods:
- **name**: The unique name of the agent.
- **description**: The description of the agent in text.
- **on_messages()**: Send the agent a sequence of ChatMessage get a Response. It is important to note that agents are expected to be stateful and this method is expected to be called with new messages, not the complete history.
- **on_messages_stream()**: Same as on_messages() but returns an iterator of AgentEvent or ChatMessage followed by a Response as the last item.
- **on_reset()**: Reset the agent to its initial state.

See autogen_agentchat.messages for more information on AgentChat message types.

## [Assistant Agent](https://microsoft.github.io/autogen/0.4.0.dev13/user-guide/agentchat-user-guide/tutorial/agents.html#assistant-agent)

AssistantAgent is a built-in agent that uses a language model and has the ability to use tools.

#### Import all the required and basic libraries

In [5]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

#### Define a basic tool to return static answers

In [2]:
# Define a tool that searches the web for information.
async def web_search(query: str) -> str:
    """Find information on the web"""
    return "AutoGen is a programming framework for building multi-agent applications."

#### To create an instance of a client to local LLM

In [18]:
# Create an agent that uses the local llm
model_client = OpenAIChatCompletionClient(
    model="ollama/llama3.2:3b",
    api_key="YOUR_API_KEY",
    base_url="http://0.0.0.0:4000/",
    model_capabilities={
        "json_output": False,
        "vision": False,
        "function_calling": True,
        },
        )

#### Define an Assistant Agent with capabilities to call the `web_search` func

In [5]:
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[web_search],
    system_message="Use tools to solve tasks.",
)

#### Get response using the `on_message` function which gives the final answer from the agentic worflow

The call to the on_messages() method returns a Response that contains the agent’s final response in the chat_message attribute, as well as a list of inner messages in the inner_messages attribute, which stores the agent’s “thought process” that led to the final response.

In [8]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="Find information on AutoGen", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(f"Internal messages >\n {response.inner_messages}")
    print(f"Final answer >\n {response.chat_message}")

In [9]:
# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()

Internal messages >
 [ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=251, completion_tokens=17), content=[FunctionCall(id='call_703b50f0-0017-4420-b052-2923e5e8fdc2', arguments='{"query": "AutoGen"}', name='web_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, content=[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.', call_id='call_703b50f0-0017-4420-b052-2923e5e8fdc2')], type='ToolCallExecutionEvent')]
Final answer >
 source='assistant' models_usage=None content='AutoGen is a programming framework for building multi-agent applications.' type='ToolCallSummaryMessage'


#### Get response using the `on_messages_stream` function which stream each message as it is generated by the agentic worflow

In [10]:
async def assistant_run_stream() -> None:
    # Option 1: read each message from the stream (as shown in the previous example).
    # async for message in agent.on_messages_stream(
    #     [TextMessage(content="Find information on AutoGen", source="user")],
    #     cancellation_token=CancellationToken(),
    # ):
    #     print(message)

    # Option 2: use Console to print all messages as they appear.
    await Console(
        agent.on_messages_stream(
            [TextMessage(content="Find information on AutoGen", source="user")],
            cancellation_token=CancellationToken(),
        )
    )


In [11]:
# Use asyncio.run(assistant_run_stream()) when running in a script.
await assistant_run_stream()

---------- assistant ----------
[FunctionCall(id='call_65c140a6-a194-40f9-9843-4030be71b36e', arguments='{"query": "AutoGen"}', name='web_search')]
[Prompt tokens: 354, Completion tokens: 18]
---------- assistant ----------
[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.', call_id='call_65c140a6-a194-40f9-9843-4030be71b36e')]
---------- assistant ----------
AutoGen is a programming framework for building multi-agent applications.
---------- Summary ----------
Number of inner messages: 2
Total prompt tokens: 354
Total completion tokens: 18
Duration: 0.34 seconds


## [Tools](https://microsoft.github.io/autogen/0.4.0.dev13/user-guide/agentchat-user-guide/tutorial/agents.html#using-tools)

To address this limitation, modern LLMs can now accept a list of available tool schemas (descriptions of tools and their arguments) and generate a tool call message. This capability is known as `Tool Calling or Function Calling` and is becoming a popular pattern in building intelligent agent-based applications.

In AgentChat, the AssistantAgent can use tools to perform specific actions. The web_search tool is one such tool that allows the assistant agent to search the web for information. A custom tool can be a Python function or a subclass of the `BaseTool`.

By default, when AssistantAgent executes a tool, it will return the tool’s output as a string in `ToolCallSummaryMessage` in its response. If your tool does not return a well-formed string in natural language, you can add a reflection step to have the model summarize the tool’s output, by setting the `reflect_on_tool_use=True` parameter in the AssistantAgent constructor.

In [127]:
from autogen_core import CancellationToken
from autogen_ext.code_executors.docker import DockerCommandLineCodeExecutor
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor
from autogen_ext.tools.code_execution import PythonCodeExecutionTool
from autogen_core.model_context import BufferedChatCompletionContext

import pandas as pd

#### Create a docker executor for the python excution tool

In [128]:
# Create the tool.
code_executor = DockerCommandLineCodeExecutor(
    work_dir="/home/ace/Everything/PlayGround/GenAI-Learnings/AutoGen_Learnings/data/code_exec/",
    )
# code_executor = LocalCommandLineCodeExecutor()
await code_executor.start()
code_execution_tool = PythonCodeExecutionTool(code_executor)
cancellation_token = CancellationToken()



#### Invoking the tool directly

In [129]:
# Use the tool directly without an agent.
code = "print('Hello, world!')"
result = await code_execution_tool.run_json({"code": code}, cancellation_token)
print(code_execution_tool.return_value_as_string(result))

Hello, world!



#### Using an assistant agent to answer the questions based on the limited history and avaiable tools

By default, AssistantAgent uses the UnboundedChatCompletionContext which sends the full conversation history to the model. To limit the context to the last n messages, you can use the BufferedChatCompletionContext.

In [130]:
agent = AssistantAgent(
    name = "assistant",
    tools=[code_execution_tool],
    model_client=model_client,
    system_message="Dataset is available to use as variable `df`. Use tools to solve tasks. Install the necessary dependencies as required.",
    model_context=BufferedChatCompletionContext(buffer_size=10)
)

df = pd.read_csv("/home/ace/Everything/PlayGround/GenAI-Learnings/AutoGen_Learnings/data/titanic.csv")

In [ ]:
await Console(
    agent.on_messages_stream(
        [TextMessage(content="What's the average age of the passengers in pandas dataframe df as defined above?", source="user")], CancellationToken()
    )
)

# User Proxy Agent

UserProxyAgent is a built-in agent that provides one way for a user to intervene in the process. This agent will put the team in a temporary blocking state, and thus any exceptions or runtime failures while in the blocked state will result in a deadlock. 

In [ ]:
from autogen_agentchat.agents import UserProxyAgent

async def user_proxy_run() -> None:
    user_proxy_agent = UserProxyAgent(
        name="user_proxy",
    )
    response = await user_proxy_agent.on_messages(
        [TextMessage(content="What is your name? ", source="user")], cancellation_token=CancellationToken()
    )
    print(f"Your name is {response.chat_message.content}")


# Use asyncio.run(user_proxy_run()) when running in a script.
await user_proxy_run()